In [5]:
import os
import sys
from dotenv import load_dotenv

# Add the parent directory to sys.path to import the geoencoding module
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from external.geoencoding import GoogleMapsGeocoder

# Initialize the geocoder
geocoder = GoogleMapsGeocoder()

# Test address
test_address = "85 division st, kingston"

try:
    # Get the geocoded result
    result_geo = geocoder.geocode_address(test_address)
    
    # Pretty print the result
    print("Geocoding Result:")
    print("-" * 50)
    for key, value in result_geo.items():
        # Format floating point numbers to 6 decimal places if they are coordinates
        if isinstance(value, float):
            print(f"{key}: {value:.6f}")
        else:
            print(f"{key}: {value}")
            
except Exception as e:
    print(f"Error: {str(e)}")


Geocoding Result:
--------------------------------------------------
address: 85 division st, kingston
latitude: 44.230387
longitude: -76.492897
formatted_address: 85 Division Street, Kingston, ON, K7L 3M1, Canada
country: Canada
province: ON
region: Frontenac County


In [6]:
from external.risk_lookup import RiskLookup

risk_lookup = RiskLookup()

test_location = risk_lookup.get_location_risks(
    {
        'country': 'Canada',
        'region': 'Frontenac County',
        'province': 'ON'
    }
)

test_location





{'Flooding': 'Relatively High',
 'Cold Wave / Severe Winter Weather': 'Relatively High',
 'Hail': 'Relatively Moderate',
 'Wildfire': 'Relatively Low',
 'Wind': 'Relatively High',
 'Earthquake': 'Relatively Moderate'}

In [3]:
from typing import Dict, Optional
from external.question_master import QuestionMaster
from external.grader import RiskGrader

# Initialize the classes
question_master = QuestionMaster()
grader = RiskGrader()

# Test QuestionMaster with Ada County, ID risks
test_risks: Dict[str, Optional[str]] = {
    'Flooding': 'Relatively High',
    'Cold Wave / Severe Winter Weather': 'Relatively High',
    'Hail': 'Relatively Low',
    'Wildfire': 'Relatively Low',
    'Wind': 'Relatively Moderate',
    'Earthquake': 'Relatively Moderate'
}

print("Testing QuestionMaster.get_relevant_questions():")
print("-" * 50)

# Get questions for high-risk categories
questions = question_master.get_relevant_questions(test_risks)

print(f"Found {len(questions)} questions for high-risk categories:")
high_risk_categories = set()
for i, q in enumerate(questions, 1):
    print(f"\n{i}. {q['question']}")
    print(f"   Risk Type: {q['risk_type']}")
    print(f"   Importance: {q['importance']}")
    high_risk_categories.add(q['risk_type'])

print(f"\nHigh risk categories covered: {', '.join(sorted(high_risk_categories))}")

# Verify that we only got questions for high-risk categories
expected_high_risk = {'Flooding', 'Winter'}  # From test_risks
assert high_risk_categories == expected_high_risk, f"Expected {expected_high_risk}, got {high_risk_categories}"


Testing QuestionMaster.get_relevant_questions():
--------------------------------------------------
Found 20 questions for high-risk categories:

1. Are all major systems (e.g. HVAC, water heater, furnace) elevated at least 1 foot above the ground?
   Risk Type: Flooding
   Importance: High

2. Do you have a sump pump installed in your home?
   Risk Type: Flooding
   Importance: High

3. Is your basement properly sealed and waterproofed? Including no cracks and leaks.
   Risk Type: Flooding
   Importance: High

4. Does your home have flood vents or openings to allow water to drain from the foundation of your home?
   Risk Type: Flooding
   Importance: Medium

5. Is the foundation of your home elevated at least 1 foot above the ground?
   Risk Type: Flooding
   Importance: High

6. Is your first floor/basement walls and flooring made with flood-resistant materials?
   Risk Type: Flooding
   Importance: Medium

7. Have all windows and doors been sealed with weather stripping and/or caulk

In [4]:
print("\nTesting RiskGrader.calculate_score() with Question-Level Scoring:")
print("-" * 50)

# Create test answers that include risk levels
test_answers = [
    # Very High Risk Questions (10 points base)
    {
        'question': 'Do you have a sump pump installed in your home?',
        'risk_type': 'Flooding',
        'importance': 'High',
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Very High'
    },
    {
        'question': 'Is your basement properly sealed and waterproofed?',
        'risk_type': 'Flooding',
        'importance': 'Medium',  # 70% multiplier
        'answer': 'No',  # Incorrect answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Very High'
    },
    # Relatively High Risk Questions (7 points base)
    {
        'question': 'Are your gutters clear of debris?',
        'risk_type': 'Winter',
        'importance': 'High',
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Relatively High'
    },
    {
        'question': 'Do you have fire resistant materials within 5 feet of your home?',
        'risk_type': 'Wildfire',
        'importance': 'Low',  # 40% multiplier
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Relatively High'
    }
]

# Calculate scores
results = grader.calculate_score(test_answers)

# Print results
print(f"\nFinal Score: {results['total_score']}%")
print(f"Points Earned: {results['points_earned']} out of {results['points_possible']}")

print("\nBreakdown by Risk Level:")
for level, stats in results['breakdown'].items():
    print(f"\n{level}:")
    print(f"  Earned: {stats['earned']} points")
    print(f"  Possible: {stats['possible']} points")
    print(f"  Score: {stats['percentage']}%")

print("\nQuestion-Level Scores (sorted by score percentage):")
print("-" * 50)
for q_score in results['question_scores']:
    print(f"\nQuestion: {q_score['question']}")
    print(f"Risk Type: {q_score['risk_type']}")
    print(f"Risk Level: {q_score['risk_level']}")
    print(f"Importance: {q_score['importance']}")
    print(f"Answer: {q_score['answer']}")
    print(f"Score: {q_score['score_percentage']:.2f}%")
    print(f"Points: {q_score['points_earned']:.2f} / {q_score['points_possible']:.2f}")
    print(f"Requires Photo: {q_score['requires_photo']}")
    print(f"Photo Validated: {q_score['photo_validated']}")

# Verify calculations
expected_points = {
    'Very High': {
        'earned': 10,  # First question correct (10 * 1.0)
        'possible': 17  # First question (10 * 1.0) + Second question (10 * 0.7)
    },
    'Relatively High': {
        'earned': 9.8,  # First question (7 * 1.0) + Second question (7 * 0.4)
        'possible': 9.8  # Same as earned since both answered correctly
    }
}

# Verify risk level totals
for level, expected in expected_points.items():
    actual = results['breakdown'][level]
    assert abs(actual['earned'] - expected['earned']) < 0.01, \
        f"{level} earned points: expected {expected['earned']}, got {actual['earned']}"
    assert abs(actual['possible'] - expected['possible']) < 0.01, \
        f"{level} possible points: expected {expected['possible']}, got {actual['possible']}"

# Verify question-level scores
expected_question_scores = [
    {
        'question': 'Is your basement properly sealed and waterproofed?',
        'points_earned': 0,
        'points_possible': 7,  # 10 base * 0.7 medium importance
        'score_percentage': 0
    },
    {
        'question': 'Do you have a sump pump installed in your home?',
        'points_earned': 10,
        'points_possible': 10,
        'score_percentage': 100
    },
    {
        'question': 'Are your gutters clear of debris?',
        'points_earned': 7,
        'points_possible': 7,
        'score_percentage': 100
    },
    {
        'question': 'Do you have fire resistant materials within 5 feet of your home?',
        'points_earned': 2.8,  # 7 base * 0.4 low importance
        'points_possible': 2.8,
        'score_percentage': 100
    }
]

# Verify each question's score details
for expected_score in expected_question_scores:
    matching_scores = [q for q in results['question_scores'] 
                      if q['question'] == expected_score['question']]
    assert len(matching_scores) == 1, f"Question not found: {expected_score['question']}"
    actual = matching_scores[0]
    
    assert abs(actual['points_earned'] - expected_score['points_earned']) < 0.01, \
        f"Points earned mismatch for {expected_score['question']}"
    assert abs(actual['points_possible'] - expected_score['points_possible']) < 0.01, \
        f"Points possible mismatch for {expected_score['question']}"
    assert abs(actual['score_percentage'] - expected_score['score_percentage']) < 0.01, \
        f"Score percentage mismatch for {expected_score['question']}"

print("\nAll assertions passed! Question-level scoring is correct.")



Testing RiskGrader.calculate_score() with Question-Level Scoring:
--------------------------------------------------

Final Score: 73.88%
Points Earned: 19.8 out of 26.8

Breakdown by Risk Level:

Very High:
  Earned: 10.0 points
  Possible: 17.0 points
  Score: 58.82%

Relatively High:
  Earned: 9.8 points
  Possible: 9.8 points
  Score: 100.0%

Question-Level Scores (sorted by score percentage):
--------------------------------------------------

Question: Is your basement properly sealed and waterproofed?
Risk Type: Flooding
Risk Level: Very High
Importance: Medium
Answer: No
Score: 0.00%
Points: 0.00 / 7.00
Requires Photo: False
Photo Validated: False

Question: Do you have a sump pump installed in your home?
Risk Type: Flooding
Risk Level: Very High
Importance: High
Answer: Yes
Score: 100.00%
Points: 10.00 / 10.00
Requires Photo: False
Photo Validated: False

Question: Are your gutters clear of debris?
Risk Type: Winter
Risk Level: Relatively High
Importance: High
Answer: Yes
Sco

In [5]:
import os
import sys
from pathlib import Path

# Add the parent directory to sys.path to import the camera module
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from external.camera import RiskPhotoManager, RiskPhotoValidator

# Initialize the classes
photo_manager = RiskPhotoManager()
validator = RiskPhotoValidator()

# Test configuration
question = "Are there any tree limbs or branches hanging over your home?"
question_id = "tree_limbs_over_home"
user_id = "test_user_123"
user_answer = "Yes"  # Based on the image, there is clearly a tree over the home
rubric = {"Yes": 0, "No": 1}  # From risk_questions.json - "Yes" indicates a risk

print("Testing RiskPhotoManager and RiskPhotoValidator:")
print("-" * 50)

# Get the absolute path to example_photo.jpg
notebook_dir = Path(os.getcwd())
photo_path = notebook_dir / "example_photo.jpg"

try:
    # Upload the photo
    with open(photo_path, "rb") as f:
        photo_data = f.read()
        photo_url = photo_manager.upload_photo(photo_data, question_id, user_id)
        
    if photo_url:
        print(f"\n✅ Successfully uploaded photo: {photo_url}")
        
        # Get all photos for this question
        photos = photo_manager.get_photos_for_question(question_id, user_id)
        print(f"\nFound {len(photos)} photos for question {question_id}")
        
        # First validate the photo quality
        validation = validator.validate_photos(photos, question)
        print("\nPhoto Validation Results:")
        print(f"Valid: {validation.get('is_valid', False)}")
        print(f"Analysis: {validation.get('analysis', 'N/A')}")
        print(f"Concerns: {', '.join(validation.get('concerns', []))}")
        
        # Then verify the answer
        verification = validator.verify_answer(photos, question, user_answer, rubric)
        print("\nAnswer Verification Results:")
        print(f"Verified: {verification.get('verified', False)}")
        print(f"Matches Answer: {verification.get('matches_answer', False)}")
        print(f"Confidence: {verification.get('confidence', 0.0)}")
        print(f"Analysis: {verification.get('analysis', 'N/A')}")
        print(f"Correct Answer: {verification.get('correct_answer', 'N/A')}")
        print(f"Evidence: {verification.get('evidence', 'N/A')}")
        print(f"Concerns: {', '.join(verification.get('concerns', []))}")
        
        # Clean up - delete the test photo
        if photo_manager.delete_photo(photo_url):
            print("\n✅ Successfully deleted test photo")
        else:
            print("\n❌ Failed to delete test photo")
            
except Exception as e:
    print(f"\n❌ Error during testing: {str(e)}")


Testing RiskPhotoManager and RiskPhotoValidator:
--------------------------------------------------
Error uploading photo: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist


In [1]:
print("\nTesting 2D to 3D Coordinate Conversion:")
print("-" * 50)

import os
import sys
import io
import requests
from pathlib import Path
from PIL import Image
from typing import Union, Optional

# Add the parent directory to sys.path to import the 2Dto3d module
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

# Import the CoordinateConverter
from external.TwoD2ThreeD import CoordinateConverter

def get_glb_from_url(url: str) -> Optional[bytes]:
    """Fetch GLB file from URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.content
    except Exception as e:
        print(f"Error fetching GLB from URL: {str(e)}")
        return None

def get_glb_content(glb_source: Union[str, bytes, Path]) -> Optional[bytes]:
    """Get GLB content from various sources."""
    try:
        if isinstance(glb_source, bytes):
            return glb_source
        elif isinstance(glb_source, str) and glb_source.startswith(('http://', 'https://')):
            return get_glb_from_url(glb_source)
        elif isinstance(glb_source, (str, Path)):
            path = Path(glb_source)
            if path.exists():
                return path.read_bytes()
        return None
    except Exception as e:
        print(f"Error reading GLB content: {str(e)}")
        return None

def run_conversion_test(glb_source: Union[str, bytes, Path], image_path: Union[str, Path]) -> None:
    """Run the conversion test with the given GLB source and image."""
    # Initialize the converter
    converter = CoordinateConverter()
    
    try:
        # Get GLB content
        glb_content = get_glb_content(glb_source)
        if not glb_content:
            raise ValueError("Failed to get GLB content")
            
        # Get image dimensions
        with Image.open(image_path) as img:
            image_dimensions = {
                'width': img.width,
                'height': img.height
            }
        
        # Create test recommendations with pixel coordinates
        test_recommendations = [
            {
                'recommendation': {
                    'question': 'Does your roof have any shingles that are missing or damaged?',
                    'risk_type': 'Winter',
                    'importance': 'High'
                },
                'location_category': 'roof',
                'pixel_coordinates': {
                    'x_pixel': int(image_dimensions['width'] * 0.5),  # Center of roof
                    'y_pixel': int(image_dimensions['height'] * 0.3)  # Upper third of image
                }
            },
            {
                'recommendation': {
                    'question': 'Is your landscaping graded away from your foundation?',
                    'risk_type': 'Flooding',
                    'importance': 'Medium'
                },
                'location_category': 'landscaping',
                'pixel_coordinates': {
                    'x_pixel': int(image_dimensions['width'] * 0.2),  # Left side of house
                    'y_pixel': int(image_dimensions['height'] * 0.8)  # Lower part of image
                }
            }
        ]

        # Create test input for the converter
        labeller_output = {
            'image_dimensions': image_dimensions,
            'labeled_recommendations': test_recommendations
        }

        # Process the conversion with GLB content
        result_3d = converter.process_labeller_output(glb_content, labeller_output)

        # Print results
        print("\nImage Dimensions:")
        print(f"Width: {image_dimensions['width']}px")
        print(f"Height: {image_dimensions['height']}px")

        print("\n3D Model Bounds:")
        bounds = result_3d['glb_model_info']['bounds_3d']
        print(f"X: {bounds['x']['min']:.3f} to {bounds['x']['max']:.3f} (range: {bounds['x']['range']:.3f})")
        print(f"Y: {bounds['y']['min']:.3f} to {bounds['y']['max']:.3f} (range: {bounds['y']['range']:.3f})")
        print(f"Z: {bounds['z']['min']:.3f} to {bounds['z']['max']:.3f} (range: {bounds['z']['range']:.3f})")

        print("\nConverted Recommendations:")
        for i, rec in enumerate(result_3d['converted_recommendations'], 1):
            print(f"\n{i}. {rec['recommendation']['question']}")
            print(f"   Category: {rec['location_category']}")
            print(f"   2D Pixel: ({rec['pixel_coordinates']['x_pixel']}, {rec['pixel_coordinates']['y_pixel']})")
            print(f"   3D Coords: ({rec['3d_coordinates']['x']:.3f}, {rec['3d_coordinates']['y']:.3f}, {rec['3d_coordinates']['z']:.3f})")
            print(f"   Z Factor: {rec['scaling_info']['z_factor']} (based on {rec['location_category']})")

        # Verify results
        assert len(result_3d['converted_recommendations']) == len(test_recommendations), \
            "Number of converted recommendations doesn't match input"
        
        for rec in result_3d['converted_recommendations']:
            # Verify coordinate existence
            assert all(key in rec['3d_coordinates'] for key in ['x', 'y', 'z']), \
                "Missing coordinate components"
            
            # Verify coordinates are within bounds
            assert bounds['x']['min'] <= rec['3d_coordinates']['x'] <= bounds['x']['max'], \
                "X coordinate out of bounds"
            assert bounds['y']['min'] <= rec['3d_coordinates']['y'] <= bounds['y']['max'], \
                "Y coordinate out of bounds"
            assert bounds['z']['min'] <= rec['3d_coordinates']['z'] <= bounds['z']['max'], \
                "Z coordinate out of bounds"
            
            # Verify z-factor assignment
            assert rec['scaling_info']['z_factor'] == converter.location_z_mapping[rec['location_category']], \
                "Incorrect z-factor assignment"

        print("\n✅ All assertions passed! 2D to 3D conversion is working correctly.")

    except FileNotFoundError as e:
        print(f"\n❌ File not found: {str(e)}")
        print("Please ensure both the image and GLB files are in the correct location.")
    except Exception as e:
        print(f"\n❌ Error during testing: {str(e)}")

# Test with different GLB sources
notebook_dir = Path(os.getcwd())
image_path = notebook_dir / "example_home.png"

# 1. Test with local file path
print("\nTesting with local GLB file:")
glb_path = notebook_dir / "model_home.glb"
run_conversion_test(glb_path, image_path)

# 2. Test with raw bytes (simulated)
print("\nTesting with raw bytes:")
try:
    with open(glb_path, 'rb') as f:
        glb_bytes = f.read()
    run_conversion_test(glb_bytes, image_path)
except FileNotFoundError:
    print("Skipping raw bytes test - GLB file not found")

# 3. Test with URL - using a public sample GLB file
print("\nTesting with URL:")
glb_url = "https://raw.githubusercontent.com/KhronosGroup/glTF-Sample-Models/master/2.0/Box/glTF-Binary/Box.glb"
run_conversion_test(glb_url, image_path)


Testing 2D to 3D Coordinate Conversion:
--------------------------------------------------

Testing with local GLB file:

Image Dimensions:
Width: 984px
Height: 676px

3D Model Bounds:
X: -0.500 to 0.500 (range: 1.000)
Y: -0.287 to 0.287 (range: 0.574)
Z: -0.499 to 0.499 (range: 0.997)

Converted Recommendations:

1. Does your roof have any shingles that are missing or damaged?
   Category: roof
   2D Pixel: (492, 202)
   3D Coords: (0.000, 0.116, 0.000)
   Z Factor: 0.5 (based on roof)

2. Is your landscaping graded away from your foundation?
   Category: landscaping
   2D Pixel: (196, 540)
   3D Coords: (-0.301, -0.172, -0.399)
   Z Factor: 0.1 (based on landscaping)

✅ All assertions passed! 2D to 3D conversion is working correctly.

Testing with raw bytes:

Image Dimensions:
Width: 984px
Height: 676px

3D Model Bounds:
X: -0.500 to 0.500 (range: 1.000)
Y: -0.287 to 0.287 (range: 0.574)
Z: -0.499 to 0.499 (range: 0.997)

Converted Recommendations:

1. Does your roof have any shingl